In [6]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import requests
from pathlib import Path  

In [7]:
url = "https://www.catch.com.au/shop/sports-outdoor/event/champion-vs-lonsdale-163891/?st=1"


In [8]:
# Use the below headers when requesting the url may help you to avoid being blocked from the site
HEADERS = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'}

#result = requests.get(url, hea)

#doc = BeautifulSoup(result.text,"html.parser")
#credits to Khuong

In [9]:
# Get the HTML content get_url()
# credits to Khuong Tran
def get_url(url):
    try:
        response = requests.get(url,headers = HEADERS).text
        soup = BeautifulSoup(response, 'html.parser')
        return soup
    except Exception as err:
        print('ERROR BY REQUEST:', err)
        return None
    

In [10]:
get_url(url).prettify()[0:1000]

'<!DOCTYPE html>\n<html class="no-js no-webp home-page" lang="en">\n <head>\n  <link as="font" crossorigin="" href="https://s.catch.com.au/static/catch/fonts/biennale_regular.woff2" rel="preload" type="font/woff2"/>\n  <link as="font" crossorigin="" href="https://s.catch.com.au/static/catch/fonts/biennale_semibold.woff2" rel="preload" type="font/woff2"/>\n  <link as="font" crossorigin="" href="https://s.catch.com.au/static/catch/fonts/biennale_black.woff2" rel="preload" type="font/woff2"/>\n  <script>\n   var cotd = cotd || {};\n  </script>\n  <script crossorigin="anonymous" src="https://polyfill.io/v3/polyfill.min.js?features=es6%2CNodeList.prototype.%40%40iterator%2CPromise%2CPromise.prototype.finally%2CglobalThis%2CObject.values%2CObject.entries%2CObject.assign%2ClocalStorage%2CArray.prototype.includes%2CArray.prototype.flatMap%2CArray.prototype.find%2CString.prototype.includes%2CString.prototype.startsWith%2CSymbol.toStringTag%2CNumber.isNaN%2Cfetch%2CElement.prototype.prepend%2Csm

In [11]:
soup = get_url(url)


In [12]:
#Obtaining main data body
data_body = soup.find_all("div" , {"class" : "product"})

#Check number of items 
print(len(data_body))

0


In [13]:
#extract item pricing
price = data_body[0].find("div" , {"class": "price--container onepass"}).get_text()

#.strip() to remove empty spaces
print(price.strip())

IndexError: list index out of range

In [ ]:
#test run for items on Sale "Don't pay XXX Save XXX etc"
testing = data_body[1].find("div" , {"class": "price--main theme"}).get_text()

testing_2 = float(testing.replace('$', ''))

testing_2

In [ ]:
#create empty dataframes to be appended on

product_name = []
product_price = []
product_savings = []
product_brands = []

for item in range(0,len(data_body)):

    #Extract Product Names
    try:
        name = data_body[item].find("a" , {"class": "js-product-link product--title-link js-dnt"}).get_text()
        product_name.append(name)  
    
    except:
        product_name.append('Empty')   
    
    #Extract Product's Brand
    try:
        brands = data_body[item].find("div" , {"class": "product--brand"}).get_text()
        product_brands.append(brands)  
    
    except:
        product_brands.append('Not Available')
    
    #Extract Product's Price
    #delete "$" and convert price to float (just incase we need to sort them)
    try:
        price = data_body[item].find("div" , {"class": "price--container onepass"}).get_text().strip()
        price = float(price.replace('$', ''))
        product_price.append(price)
    
    #Code will run into an error if an item is on sale because price will be assigned to "price--main theme"
    #This code will extract price data from price--main theme instead
    except:
        price2 = data_body[item].find("div" , {"class": "price--main theme"}).get_text()
        price2 = float(price2.replace('$', ''))
        product_price.append(price2)
     
    #Extract Product's Savings   
    try:
        savings = data_body[item].find("span" , {"data-ts": "savings-value"}).get_text()
        product_savings.append(savings)
        
    except:
        product_savings.append('-')      

d = {'Product Name': product_name, 'Brand': product_brands, 'Price ($AUD)': product_price, "Savings!": product_savings}

cotd_df = pd.DataFrame(data=d)

cotd_df

In [ ]:
#export to csv
filepath = Path('..\Web-Scraping-Project\catch_of_the_day.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
cotd_df.to_csv(filepath)